Aquí vamos a hacer una coleccion de errores para el conjunto total de test y lo vamos a guardar en un archivo excel o csv para futuras lecturas.

In [11]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils_nn as utlnn


**Nota**: Uso de funciones Personalizadas con tf.map_fn.
Al tener funciones personalizadas, podemos usar tf.map_fn para aplicar a cada elemento del tensor.

In [12]:
x_test = pd.read_csv("./Train_Test/x_test.csv")
y_test = pd.read_csv("./Train_Test/y_test.csv")

autoencoder_Lat_Lon = tf.keras.models.load_model('modelos_entrenamiento/modelos_latitudes_longitudes/entrenamiento_lat_lon_600_120.keras')
autoencoder_Height = tf.keras.models.load_model('modelos_entrenamiento/entrenamiento_alturas2.keras')

In [13]:
lat_pred, lon_pred = autoencoder_Lat_Lon.predict(x_test)
heights_pred = autoencoder_Height.predict(x_test)


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
# lat_pred[0]

In [15]:
y_true_lat = y_test.iloc[:,:100]
y_true_lon = y_test.iloc[:,100:200]
y_true_heights = y_test.iloc[:,200:300] #km
print("Formatos True y Pred: ",type(y_true_lat), type(lat_pred))
y_true_lat = y_true_lat.to_numpy()
y_true_lon = y_true_lon.to_numpy()
y_true_heights = y_true_heights.to_numpy()
print("Formatos True y Pred: ",type(y_true_heights), type(heights_pred))


Formatos True y Pred:  <class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>
Formatos True y Pred:  <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [16]:
distancias_2d = np.array(list(map(lambda args: np.array(utlnn.distances_by_geodesic(*args)), zip(y_true_lat, y_true_lon, lat_pred, lon_pred))))

In [17]:
distancias_3d = np.array(list(map(lambda args: np.array(utlnn.distances_3D(*args)), zip(distancias_2d,y_true_heights,heights_pred))))

In [18]:
error_2d = np.array(list(map(lambda args: np.array(utlnn.ecm_recm(args)),distancias_2d)))

In [19]:
error_3d =  np.array(list(map(lambda args: np.array(utlnn.ecm_recm(args)),distancias_3d)))

In [20]:
save_errors = True
nameFile = "errors"
sheet_name = "lat_lon_600_120_alturas2"

if save_errors == True:
  new_df= utlnn.generate_df(distancias_2d,error_2d,distancias_3d,error_3d)
  utlnn.save_file_error(new_df,nameFile,sheet_name)